In [1]:
rule = ".#./..#/###"

In [2]:
def pp(pattern):
    for row in pattern:
        print(row)

In [3]:
pattern = rule.split('/')
pp(pattern)

.#.
..#
###


In [4]:
def mirror(pattern):
    """mirror pattern"""
    return [row[::-1] for row in pattern]

pp(mirror(pattern))

.#.
#..
###


In [5]:
def rotate(pattern):
    """rotate 90 deg CW"""
    return [''.join(row) for row in zip(*pattern[::-1])]

pp(rotate(pattern))

#..
#.#
##.


In [6]:
def combinations(p):
    """yield all combinations
    3 rotations and 3 rotations + mirror

    """
    for _ in range(4):
        yield p
        yield mirror(p)
        p = rotate(p)
    

In [7]:
# check combinations
uniq = set()
for i, p in enumerate(combinations(pattern), 1):
    pp(p)
    print()
    uniq.add(str(p))
    assert i == len(uniq)
assert len(uniq) == 8

.#.
..#
###

.#.
#..
###

#..
#.#
##.

..#
#.#
.##

###
#..
.#.

###
..#
.#.

.##
#.#
..#

##.
#.#
#..



In [8]:
#book = ['../.# => ##./#../...', '.#./..#/### => #..#/..../..../#..#']
f = open('input\input21.txt')
book = f.readlines()
rules = [rule.strip().split(' => ') for rule in book]
rules[:3]

[['../..', '..#/#../.#.'], ['#./..', '#../#../...'], ['##/..', '###/#.#/#..']]

In [9]:
rules_dict = {}
for src, dst in rules:
    new_pattern = dst.split('/')
    print(src, ' --> ', new_pattern)

    for pattern in combinations(src.split('/')):
        rules_dict[str(pattern)] = new_pattern


../..  -->  ['..#', '#..', '.#.']
#./..  -->  ['#..', '#..', '...']
##/..  -->  ['###', '#.#', '#..']
.#/#.  -->  ['###', '##.', '.#.']
##/#.  -->  ['...', '.#.', '..#']
##/##  -->  ['##.', '#.#', '###']
.../.../...  -->  ['##..', '.#..', '#.#.', '....']
#../.../...  -->  ['....', '##.#', '...#', '##.#']
.#./.../...  -->  ['###.', '####', '#...', '#..#']
##./.../...  -->  ['###.', '.##.', '...#', '..##']
#.#/.../...  -->  ['.###', '.##.', '#...', '#.##']
###/.../...  -->  ['##.#', '#..#', '#.#.', '#.##']
.#./#../...  -->  ['#.#.', '.###', '#...', '#.##']
##./#../...  -->  ['#...', '####', '#.##', '....']
..#/#../...  -->  ['#.##', '..#.', '...#', '...#']
#.#/#../...  -->  ['#.##', '####', '.#.#', '#.#.']
.##/#../...  -->  ['#...', '##..', '##.#', '.##.']
###/#../...  -->  ['....', '#.#.', '.###', '#...']
.../.#./...  -->  ['.#.#', '#..#', '##..', '#.##']
#../.#./...  -->  ['###.', '.###', '.#.#', '..#.']
.#./.#./...  -->  ['..##', '.##.', '..##', '.#.#']
##./.#./...  -->  ['..#.', '##.

In [10]:
list(rules_dict.items())[:5]

[("['..', '..']", ['..#', '#..', '.#.']),
 ("['#.', '..']", ['#..', '#..', '...']),
 ("['.#', '..']", ['#..', '#..', '...']),
 ("['..', '.#']", ['#..', '#..', '...']),
 ("['..', '#.']", ['#..', '#..', '...'])]

In [11]:
new_pattern = rules_dict[str(pattern)]
assert len(new_pattern) == 4

In [12]:
p4 = ['1#..', '.2..', '.#3.', '...4']
pp(p4)

1#..
.2..
.#3.
...4


In [13]:
p6 = ['1###!!', '.2..!!', '.#3#..', '...4..', '...#5.', '...##6']
pp(p6)

1###!!
.2..!!
.#3#..
...4..
...#5.
...##6


In [14]:
# this is why numpy was invented...
def split(p, n=3):
    """split array into nxn parts"""
    for r in range(0, len(p), n):
        for c in range(0, len(p), n):
            yield [x[c:c+n] for x in p[r:r+n]] 
        
for m in split(p6, 3):
    pp(m)

1##
.2.
.#3
#!!
.!!
#..
...
...
...
4..
#5.
##6


In [15]:
l = [x for x in split(p6, 3)]
l

[['1##', '.2.', '.#3'],
 ['#!!', '.!!', '#..'],
 ['...', '...', '...'],
 ['4..', '#5.', '##6']]

In [16]:
def rejoin(l):
    """join list of patterns in square nxn matrix"""
    assert len(l) == 4
    p1, p2, p3, p4 = l
    return [p1[0]+p2[0], p1[1]+p2[1], p1[2]+p2[2],
            p3[0]+p4[0], p3[1]+p4[1], p3[2]+p4[2]]

pp(rejoin(l))
assert rejoin(l) == p6

1###!!
.2..!!
.#3#..
...4..
...#5.
...##6


In [17]:
import functools

def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

@memoize
def expand3x3(pattern, n=3):
    """expand a single 3x3 tile into 9 3x3 tiles"""
    
    if n==1:
        assert False  # Not Implemented
        
    assert len(pattern) == 3
    
    # step 1 evenly divisible by 3:
    p4 = rules_dict[str(pattern)]
    assert len(p4) == 4
    
    # step 2 evenly divisible by 2: break into 2x2 and expand to 3x3
    p6 = [rules_dict[str(x)] for x in split(p4, 2)]
    assert len(p6) == 4
    assert len(p6[0]) == 3
   
    if n == 2:
        return p6
    
    # step 3 evenly divisible by 2: rejoin to 6x6, break into 3x3:
    p9 = rejoin(p6)
    return [rules_dict[str(x)] for x in split(p9, 2)]

expand3x3(rule.split('/'), n=2)
    

[['##.', '#.#', '###'],
 ['...', '.#.', '..#'],
 ['#..', '#..', '...'],
 ['...', '.#.', '..#']]

In [18]:
def solve(n=5):
    start = ".#./..#/###".split('/')
    queue = [start]
    for _ in range(n//3):
        next_queue = []
        for tile in queue:
            next_queue.extend(expand3x3(tile))
        print('iteration: ', _ ,' ', len(next_queue))
        queue = next_queue
        
    # remainder of iterations:
    if n % 3:
        next_queue = []
        for tile in queue:
            next_queue.extend(expand3x3(tile, n=(n % 3))) 
    
    return next_queue

solve(n=5)
    

iteration:  0   9


[['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['#..', '#..', '...'],
 ['###', '##.', '.#.'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['..#', '#..', '.#.'],
 ['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['..#', '#..', '.#.'],
 ['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['#..', '#..', '...'],
 ['###', '##.', '.#.'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['..#', '#..', '.#.'],
 ['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['..#', '#..', '.#.'],
 ['...', '.#.', '..#'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['..#', '#..', '.#.'],
 ['...', '.#.', '..#'],
 ['#..', '#..', '...'],
 ['...', '.#.', '..#'],
 ['###', '#.#', '#..'],
 ['...', '.#.', '..#'],
 ['###', '##.', '.#.'],
 ['#..', '#..', '...'],
 ['###', '#.#', '#..'],
 ['...', '.#.', '..#']]

In [20]:
# part A

In [21]:
result = solve(n=5)

iteration:  0   9


In [22]:
n = 0
for tile in result:
    for row in tile:
        n += row.count('#')
n

117

In [23]:
#part B

In [24]:
result = solve(n=18)

iteration:  0   9
iteration:  1   81
iteration:  2   729
iteration:  3   6561
iteration:  4   59049
iteration:  5   531441


In [25]:
n = 0
for tile in result:
    for row in tile:
        n += row.count('#')
n

2026963